ETL

In [2]:
#librerias
import pandas as pd
import numpy as np


Se descargo los data set en formato .xlsx algunos de estos tiene varias hojas por data set  para realizar el respectivo análisis debemos convertimos a formato csv 


Cargamos el data set Internet.xlsx para extraer las diferentes hojas que lo conforman y las trasformamos a formato .csv y las guardamos en la carpeta internet

In [4]:
# Cargamos el data set Internet en formato .xlsx
excel_file =r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .xlsx/Internet.xlsx'

# Leemos todas las hojas del archivo Excel
hojas = pd.read_excel(excel_file, sheet_name=None)

# Convertimos cada hoja a un archivo CSV
csv_files = []
for nombre_hoja, datos_hoja in hojas.items():
    # Definmos el nombre del archivo CSV basado en el nombre de la hoja
    nombre_csv = f"H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/{nombre_hoja}.csv"
    # Guardamos cada hoja en un archivo CSV
    datos_hoja.to_csv(nombre_csv, index=False)
    csv_files.append(nombre_csv)
    print(f"La hoja de nombre '{nombre_hoja}' a sido combredtida aconvertida a {nombre_csv}")


La hoja de nombre 'Acc_vel_loc_sinrangos' a sido combredtida aconvertida a H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Acc_vel_loc_sinrangos.csv
La hoja de nombre 'Velocidad_sin_Rangos' a sido combredtida aconvertida a H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Velocidad_sin_Rangos.csv
La hoja de nombre 'Accesos_tecnologia_localidad' a sido combredtida aconvertida a H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Accesos_tecnologia_localidad.csv
La hoja de nombre 'Velocidad % por prov' a sido combredtida aconvertida a H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Velocidad % por prov.csv
La hoja de nombre 'Totales VMD' a sido combredtida aconvertida a H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Totales VMD.csv
La hoja de nombre 'Totales Accesos Por Tecnología' a sido combredtida aconvertida a H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Totales Accesos Por Tecnología.csv
La hoja de nombre 'Accesos Por Tecnología' a sido combre

ETL Acc_vel_loc_sinrangos

Cargamos el data set Internet .csv y realizamos una exploración para ver datos faltantes ,su estructura y visualizar las primeras líneas

In [5]:
# Cargamos el archivo CSV
archivo= r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Acc_vel_loc_sinrangos.csv'
print(f"Análisis para el archivo: {archivo}")
df = pd.read_csv(archivo)

 # Realizamos el analisis  de el archivo
print(df.head())         
print(df.info())        
print(df.describe())     
print(f"Dimensiones del dataframe: {df.shape}")

# Análisamos el porsentaje de valores nulos valores nulos
print("Valores nulos por columna")
print(df.isnull().sum())

# Contamos el valores faltantes
missing_values = df.isnull().mean() * 100
print("Porcentaje de valores faltantes por columna:\n", missing_values)

# Contamos el número de filas duplicadas
num_duplicates = df.duplicated(keep=False).sum()
print("Número total de filas duplicadas:", num_duplicates)
print("\n" + "="*50 + "\n")  # Separador para cada archivo

Análisis para el archivo: H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Acc_vel_loc_sinrangos.csv
        Partido   Localidad  link Indec Velocidad (Mbps)  Provincia  Accesos
0  BUENOS AIRES  25 de Mayo  25 de Mayo          6854100       0.00      1.0
1  BUENOS AIRES  25 de Mayo  25 de Mayo          6854100       0.50      2.0
2  BUENOS AIRES  25 de Mayo  25 de Mayo          6854100       0.75     19.0
3  BUENOS AIRES  25 de Mayo  25 de Mayo          6854100       3.00     85.0
4  BUENOS AIRES  25 de Mayo  25 de Mayo          6854100       3.50    145.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18864 entries, 0 to 18863
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Partido           18864 non-null  object 
 1   Localidad         18864 non-null  object 
 2   link Indec        18863 non-null  object 
 3   Velocidad (Mbps)  18864 non-null  object 
 4   Provincia         18864 non-null  fl

Una ves detectado si hay valores faltantes los cambiaremos por none  

In [6]:
# Cambiamos valores faltantes por None
df = df.where(pd.notnull(df), None)
df.replace(r"^\s*$", None, regex=True, inplace=True)


# Análisis de valores nulos
print("Valores nulos por columna")
print(df.isnull().sum())


Valores nulos por columna
Partido             0
Localidad           0
link Indec          1
Velocidad (Mbps)    0
Provincia           0
Accesos             7
dtype: int64


Se crea un columna id Provincia y Id Localidad se elimina las columnas link id(representan pequeñas localidades no las considero relevantes) y provincia(No  encuentro que representa por eso la veo de baja relevancia),además se ordena las colunas en base a su relevancia
En  columna localidad se repiten las localidades por lo que sumamos los datos de acceso y promediamos los de velocidad 

In [7]:
# Cargamos el archivo CSV
archivo = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Acc_vel_loc_sinrangos.csv'
df = pd.read_csv(archivo)

# Eliminamos columnas poco relevantes
df.drop(columns=['Provincia', 'link Indec'], inplace=True)

# Reemplazamos valores vacíos en 'Accesos' con 0 o NULL
df['Accesos'] = df['Accesos'].fillna(0)  # Cambia a None si deseas NULL

# Cambiamos el nombre de la columna Partido a Provincia
df.rename(columns={'Partido': 'Provincia'}, inplace=True)

# Identificamos todas las provincias en el dataset
provincias = df['Provincia'].unique()
print("Provincias en Argentina:", provincias)
print("Número total de provincias:", len(provincias))

# Creamos un diccionario de ID únicos para cada provincia, comenzando desde 1
provincia_id_map = {provincia: idx + 1 for idx, provincia in enumerate(provincias)}

# Creamos la columna Id_Provincia usando el diccionario
df['Id_Provincia'] = df['Provincia'].map(provincia_id_map)

# Identificamos todas las localidades en el dataset
localidades = df['Localidad'].unique()
print("Número total de localidades:", len(localidades))

# Creamos un diccionario único para cada localidad comenzando en 1
localidad_id_map = {localidad: idx + 1 for idx, localidad in enumerate(localidades)}

# Creamos la columna Id_Localidad usando el diccionario
df['Id_Localidad'] = df['Localidad'].map(localidad_id_map)

# Ordenamos el dataset según el orden de columnas
column_order = ['Id_Provincia', 'Provincia', 'Id_Localidad', 'Localidad', 'Velocidad (Mbps)', 'Accesos']
df = df[column_order]

# Configuramos pandas para mostrar todos los dígitos en la salida sin notación científica
pd.options.display.float_format = '{:.1f}'.format


#Nos aseguramos que todos los datos de la columna velocidad se encuentren en un formato numerico
df['Velocidad (Mbps)'] = pd.to_numeric(df['Velocidad (Mbps)'], errors='coerce')

# Agrupamos y calculamos la suma de  datos de la columna 'Accesos' y el promedio de la columna  'Velocidad (Mbps)', redondeando a 1 decimal .Todo esto lo asemos por localidad
df_grouped = df.groupby(['Id_Provincia', 'Provincia', 'Id_Localidad', 'Localidad'], as_index=False).agg({
    'Accesos': 'sum',
    'Velocidad (Mbps)': 'mean'
})
df_grouped['Velocidad (Mbps)'] = df_grouped['Velocidad (Mbps)'].round(1)


# Guardamos el DataFrame final en un archivo CSV
output_path = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Acc_vel_loc_sinrangos.csv'
df.to_csv(output_path, index=False)

print(df.head())


Provincias en Argentina: ['BUENOS AIRES' 'CABA' 'CATAMARCA' 'CHACO' 'CHUBUT' 'CORDOBA' 'CORRIENTES'
 'ENTRE RIOS' 'FORMOSA' 'JUJUY' 'LA PAMPA' 'LA RIOJA' 'MENDOZA' 'MISIONES'
 'NEUQUEN' 'RIO NEGRO' 'SALTA' 'SAN JUAN' 'SAN LUIS' 'SANTA CRUZ'
 'SANTA FE' 'SANTIAGO DEL ESTERO' 'TIERRA DEL FUEGO' 'TUCUMAN']
Número total de provincias: 24
Número total de localidades: 435
   Id_Provincia     Provincia  Id_Localidad   Localidad  Velocidad (Mbps)  \
0             1  BUENOS AIRES             1  25 de Mayo         6854100.0   
1             1  BUENOS AIRES             1  25 de Mayo         6854100.0   
2             1  BUENOS AIRES             1  25 de Mayo         6854100.0   
3             1  BUENOS AIRES             1  25 de Mayo         6854100.0   
4             1  BUENOS AIRES             1  25 de Mayo         6854100.0   

   Accesos  
0      1.0  
1      2.0  
2     19.0  
3     85.0  
4    145.0  


ETL rangos.csv

Cargamos el data set Accesos por rangos.csv y realizamos una exploración para ver datos faltantes ,su estructura y visualizar las primeras líneas

In [8]:
# Cargamos el archivo CSV
archivo= r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Accesos por rangos.csv'
print(f"Análisis para el archivo: {archivo}")  # Muestra el nombre del archivo
df = pd.read_csv(archivo)

 # Realizamos el analisis  de el archivo
print(df.head())         
print(df.info())        
print(df.describe())     
print(f"Dimensiones del dataframe: {df.shape}")

# Análisamos el porsentaje de valores nulos valores nulos
print("Valores nulos por columna")
print(df.isnull().sum())

# Contamos el valores faltantes
missing_values = df.isnull().mean() * 100
print("Porcentaje de valores faltantes por columna:\n", missing_values)

# Contamos el número de filas duplicadas
num_duplicates = df.duplicated(keep=False).sum()
print("Número total de filas duplicadas:", num_duplicates)
print("\n" + "="*50 + "\n")  # Separador para cada archivo


Análisis para el archivo: H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Accesos por rangos.csv
    Año  Trimestre        Provincia  HASTA 512 kbps  + 512 Kbps - 1 Mbps  \
0  2024          2     Buenos Aires         25287.0              23034.0   
1  2024          2  Capital Federal           518.0               4122.0   
2  2024          2        Catamarca            72.0                164.0   
3  2024          2            Chaco           236.0                349.0   
4  2024          2           Chubut           125.0               1139.0   

   + 1 Mbps - 6 Mbps  + 6 Mbps - 10 Mbps  + 10 Mbps - 20 Mbps  \
0           222627.0            221660.0             244528.0   
1            24539.0             40296.0              33158.0   
2             2660.0              3093.0               3799.0   
3            14594.0              8336.0               6208.0   
4            43699.0             31971.0              22696.0   

   + 20 Mbps - 30 Mbps  + 30 Mbps    OTROS     T

Cambiamos el formato de fecha para que sea un formato mas compatible con Python uniendo la columna Año con la columna trimestre  para ser remplasadas por Fecha y se crea la  columna Id_provincia

In [10]:
# Cargamos el data set 
file_path = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Accesos por rangos.csv'
df = pd.read_csv(file_path)

# Mapeamos los trimestres al último día de cada trimestre
trimestre_map = {1: "03-31", 2: "06-30", 3: "09-30", 4: "12-31"}

# Convertimos 'Año' a cadena y mapear 'Trimestre' al último día del trimestre
if 'Año' in df.columns and 'Trimestre' in df.columns:
    df['Año'] = df['Año'].astype(str)
    df['Trimestre'] = df['Trimestre'].map(trimestre_map)

    # Creamos la columna 'Fecha' combinando 'Año' y 'Trimestre'
    df['Fecha'] = pd.to_datetime(df['Año'] + "-" + df['Trimestre'])

    # Borramos  las columnas 'Año' y 'Trimestre'
    df.drop(columns=['Año', 'Trimestre'], inplace=True)

    # Reordenamos las columnas 
    cols = ['Fecha'] + [col for col in df.columns if col != 'Fecha']
    df = df[cols]
else:
    print("Error: Las columnas 'Año' y 'Trimestre' no se encontraron en los datos.")

# Identificamos todas las provincias en el dataset
provincias = df['Provincia'].unique()
print("Provincias en Argentina:", provincias)
print("Número total de provincias:", len(provincias))

# Creamos un diccionario de ID únicos para cada provincia, comenzando desde 1
provincia_id_map = {provincia: idx + 1 for idx, provincia in enumerate(provincias)}

# Creamos la columna Id_Provincia usando el diccionario
df['Id_Provincia'] = df['Provincia'].map(provincia_id_map)

# Ordenamos el dataset según el orden de columnas
column_order = ['Id_Provincia','Provincia', 'Fecha','HASTA 512 kbps','+ 512 Kbps - 1 Mbps','+ 1 Mbps - 6 Mbps','+ 6 Mbps - 10 Mbps','+ 10 Mbps - 20 Mbps','+ 20 Mbps - 30 Mbps','+ 30 Mbps','OTROS','Total']
df = df[column_order]

print(df.head()) 

# Guardar el DataFrame final en un archivo CSV
output_path = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Accesos por rangos.csv'
df.to_csv(output_path, index=False)

# Confirmación de guardado
print(f"Archivo se guardo correctamente en: {output_path}")



Error: Las columnas 'Año' y 'Trimestre' no se encontraron en los datos.
Provincias en Argentina: ['Buenos Aires' 'Capital Federal' 'Catamarca' 'Chaco' 'Chubut' 'Córdoba'
 'Corrientes' 'Entre Ríos' 'Formosa' 'Jujuy' 'La Pampa' 'La Rioja'
 'Mendoza' 'Misiones' 'Neuquén' 'Río Negro' 'Salta' 'San Juan' 'San Luis'
 'Santa Cruz' 'Santa Fe' 'Santiago Del Estero' 'Tierra Del Fuego'
 'Tucumán']
Número total de provincias: 24
   Id_Provincia        Provincia       Fecha  HASTA 512 kbps  \
0             1     Buenos Aires  2024-06-30         25287.0   
1             2  Capital Federal  2024-06-30           518.0   
2             3        Catamarca  2024-06-30            72.0   
3             4            Chaco  2024-06-30           236.0   
4             5           Chubut  2024-06-30           125.0   

   + 512 Kbps - 1 Mbps  + 1 Mbps - 6 Mbps  + 6 Mbps - 10 Mbps  \
0              23034.0           222627.0            221660.0   
1               4122.0            24539.0             40296.0   


ETL Tecnología.csv 

Cargamos el data set Accesos Por Tecnología.csv y realizamos una exploración para ver datos faltantes ,su estructura y visualizar las primeras líneas

In [11]:
# Cargamos el archivo CSV
archivo= r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Accesos Por Tecnología.csv'
print(f"Análisis para el archivo: {archivo}")  

 # Realizamos el analisis  de el archivo
print(df.head())         
print(df.info())        
print(df.describe())     
print(f"Dimensiones del dataframe: {df.shape}")

# Análisamos el porsentaje de valores nulos valores nulos
print("Valores nulos por columna")
print(df.isnull().sum())

# Contamos el valores faltantes
missing_values = df.isnull().mean() * 100
print("Porcentaje de valores faltantes por columna:\n", missing_values)

# Contamos el número de filas duplicadas
num_duplicates = df.duplicated(keep=False).sum()
print("Número total de filas duplicadas:", num_duplicates)
print("\n" + "="*50 + "\n")  # Separador para cada archivo

Análisis para el archivo: H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Accesos Por Tecnología.csv
   Id_Provincia        Provincia       Fecha  HASTA 512 kbps  \
0             1     Buenos Aires  2024-06-30         25287.0   
1             2  Capital Federal  2024-06-30           518.0   
2             3        Catamarca  2024-06-30            72.0   
3             4            Chaco  2024-06-30           236.0   
4             5           Chubut  2024-06-30           125.0   

   + 512 Kbps - 1 Mbps  + 1 Mbps - 6 Mbps  + 6 Mbps - 10 Mbps  \
0              23034.0           222627.0            221660.0   
1               4122.0            24539.0             40296.0   
2                164.0             2660.0              3093.0   
3                349.0            14594.0              8336.0   
4               1139.0            43699.0             31971.0   

   + 10 Mbps - 20 Mbps  + 20 Mbps - 30 Mbps  + 30 Mbps    OTROS     Total  
0             244528.0             10630

Cambiamos el formato de fecha para que sea un formato mas compatible con Python uniendo la columna fecha con la columna trimestre , ademas creamos la columna Id_Provincia

In [12]:
import pandas as pd

# Cargar el dataset
file_path = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Accesos Por Tecnología.csv'
df = pd.read_csv(file_path)

#Mapeamos los trimestres al último día de cada trimestre
trimestre_map = {2: "06-30", 1: "03-31", 4: "12-31", 3: "09-30"}
print("Valores únicos en Trimestre antes de limpieza:", df['Trimestre'].unique())

# Limpiamos la columna 'Trimestre' eliminando caracteres no numéricos
df['Trimestre'] = df['Trimestre'].astype(str).str.extract(r'(\d)').astype(float)

# Limpiamos la columna 'Año' eliminando caracteres no numéricos
df['Año'] = df['Año'].astype(str).str.extract(r'(\d{4})')

df['Año'] = df['Año'].fillna(0).astype(int)  # Sustituir valores NaN por un valor predeterminado
df['Trimestre'] = df['Trimestre'].fillna(0).astype(float)

# Convertimos 'Año' a cadena y mapear 'Trimestre' al último día del trimestre
if 'Año' in df.columns and 'Trimestre' in df.columns:
    df['Año'] = df['Año'].astype(str)
    df['Trimestre'] = df['Trimestre'].map(trimestre_map)

    # Creamos la columna 'Fecha' combinando 'Año' y 'Trimestre'
    df['Fecha'] = pd.to_datetime(df['Año'] + "-" + df['Trimestre'], errors='coerce')

    # Eliminamos las columnas 'Año' y 'Trimestre'
    df.drop(columns=['Año', 'Trimestre'], inplace=True)

    # Reordenamos las columnas
    cols = ['Fecha'] + [col for col in df.columns if col != 'Fecha']
    df = df[cols]
else:
    print("Error: Las columnas 'Año' y 'Trimestre' no se encontraron en los datos.")

# Identificamos todas las provincias en el dataset
provincias = df['Provincia'].unique()
print("Provincias en Argentina:", provincias)
print("Número total de provincias:", len(provincias))

# Creaamos un diccionario de ID únicos para cada provincia, comenzando desde 1
provincia_id_map = {provincia: idx + 1 for idx, provincia in enumerate(provincias)}

# Creamos la columna Id_Provincia usando el diccionario
df['Id_Provincia'] = df['Provincia'].map(provincia_id_map)

# Ordenamos el dataset según el orden de columnas
column_order = ['Id_Provincia', 'Provincia', 'Fecha', 'ADSL', 'Cablemodem', 'Fibra óptica', 'Wireless', 'Otros', 'Total']
df = df[column_order]

# Guardamos el DataFrame final en un archivo CSV
output_path = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Accesos Por Tecnología.csv'
df.to_csv(output_path, index=False)

print(df.head())




Valores únicos en Trimestre antes de limpieza: ['2' '1' '4' '3' '3 *' '2 *' '1 *' nan
 'Los datos provinciales no coinciden a nivel nacional, ya que se rincorporó información que no contien apertuta a nivel geográfico.']
Provincias en Argentina: ['Buenos Aires' 'Capital Federal' 'Catamarca' 'Chaco' 'Chubut' 'Córdoba'
 'Corrientes' 'Entre Ríos' 'Formosa' 'Jujuy' 'La Pampa' 'La Rioja'
 'Mendoza' 'Misiones' 'Neuquén' 'Río Negro' 'Salta' 'San Juan' 'San Luis'
 'Santa Cruz' 'Santa Fe' 'Santiago Del Estero' 'Tierra Del Fuego'
 'Tucumán' nan]
Número total de provincias: 25
   Id_Provincia        Provincia      Fecha     ADSL  Cablemodem  \
0             1     Buenos Aires 2024-06-30 214055.0   2722466.0   
1             2  Capital Federal 2024-06-30  54102.0   1144781.0   
2             3        Catamarca 2024-06-30   4951.0     10303.0   
3             4            Chaco 2024-06-30   9448.0     57935.0   
4             5           Chubut 2024-06-30  25955.0     80704.0   

   Fibra óptica  W

ETL  Accesos_tecnologia_localidad.csv

Cargamos el data set Accesos_tecnologia_localidad.csv y realizamos una exploración para ver datos faltantes ,su estructura y visualizar las primeras líneas

In [13]:
# Cargamos el archivo CSV
archivo= r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Accesos_tecnologia_localidad.csv'
print(f"Análisis para el archivo: {archivo}") 
df = pd.read_csv(archivo)

 # Realizamos el analisis  de el archivo
print(df.head())         
print(df.info())        
print(df.describe())     
print(f"Dimensiones del dataframe: {df.shape}")

# Análisamos el porsentaje de valores nulos valores nulos
print("Valores nulos por columna")
print(df.isnull().sum())

# Contamos el valores faltantes
missing_values = df.isnull().mean() * 100
print("Porcentaje de valores faltantes por columna:\n", missing_values)

# Contamos el número de filas duplicadas
num_duplicates = df.duplicated(keep=False).sum()
print("Número total de filas duplicadas:", num_duplicates)
print("\n" + "="*50 + "\n")  # Separador para cada archivo

Análisis para el archivo: H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Accesos_tecnologia_localidad.csv
      Provincia     Partido   Localidad    Tecnologia Link Indec  Accesos
0  BUENOS AIRES  25 de Mayo  25 de Mayo          ADSL    6854100    755.0
1  BUENOS AIRES  25 de Mayo  25 de Mayo    CABLEMODEM    6854100   4600.0
2  BUENOS AIRES  25 de Mayo  25 de Mayo  FIBRA OPTICA    6854100      2.0
3  BUENOS AIRES  25 de Mayo  25 de Mayo     SATELITAL    6854100    742.0
4  BUENOS AIRES  25 de Mayo  25 de Mayo      WIRELESS    6854100    727.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7753 entries, 0 to 7752
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Provincia   7753 non-null   object 
 1   Partido     7753 non-null   object 
 2   Localidad   7753 non-null   object 
 3   Tecnologia  7753 non-null   object 
 4   Link Indec  7753 non-null   object 
 5   Accesos     7747 non-null   float64
dtypes: 

Se crea un columna id_Provincia , Id_Partido  y Id_Localidad 

In [14]:
# Cargamos el archivo CSV
archivo = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Accesos_tecnologia_localidad.csv'
df = pd.read_csv(archivo)

# Identificamos todas las provincias en el dataset
provincias = df['Provincia'].unique()
print("Provincias en Argentina:", provincias)
print("Número total de provincias:", len(provincias))

# Creamos un diccionario de ID únicos para cada provincia, comenzando desde 1
provincia_id_map = {provincia: idx + 1 for idx, provincia in enumerate(provincias)}

# Creamos la columna Id_Provincia usando el diccionario
df['Id_Provincia'] = df['Provincia'].map(provincia_id_map)


# Reemplazamos valores vacíos o espacios en blanco en la columna 'Accesos' con 0 
df['Accesos'] = pd.to_numeric(df['Accesos'], errors='coerce') 
df['Accesos'].fillna(0, inplace=True)  

# Identificamos todas los Partido en el dataset
partido = df['Partido'].unique()
print("Número total de Partido:", len(partido))

# Creamos un diccionario único para cada partido comenzando en 1
partido_id_map = {partido: idx + 1 for idx, partido in enumerate(partido)}

# Creamos la columna Id_Localidad usando el diccionario
df['Id_Partido'] = df['Partido'].map(partido_id_map)


# Identificamos todas las localidades en el dataset
localidades = df['Localidad'].unique()
print("Número total de localidades:", len(localidades))

# Creamos un diccionario único para cada localidad comenzando en 1
localidad_id_map = {localidad: idx + 1 for idx, localidad in enumerate(localidades)}

# Creamos la columna Id_Localidad usando el diccionario
df['Id_Localidad'] = df['Localidad'].map(localidad_id_map)


# Ordenamos el dataset según el orden de columnas
column_order = ['Id_Provincia','Provincia', 'Id_Partido','Partido','Id_Localidad','Localidad' ,'Tecnologia','Link Indec','Accesos']
df = df[column_order]

# Guardamos el DataFrame final en un archivo CSV
output_path = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Accesos_tecnologia_localidad.csv'
df.to_csv(output_path, index=False)

print(df.head())






Provincias en Argentina: ['BUENOS AIRES' 'CABA' 'CATAMARCA' 'CHACO' 'CHUBUT' 'CORDOBA' 'CORRIENTES'
 'ENTRE RIOS' 'FORMOSA' 'JUJUY' 'LA PAMPA' 'LA RIOJA' 'MENDOZA' 'MISIONES'
 'NEUQUEN' 'RIO NEGRO' 'SALTA' 'SAN JUAN' 'SAN LUIS' 'SANTA CRUZ'
 'SANTA FE' 'SANTIAGO DEL ESTERO' 'TIERRA DEL FUEGO' 'TUCUMAN']
Número total de provincias: 24
Número total de Partido: 435
Número total de localidades: 2793
   Id_Provincia     Provincia  Id_Partido     Partido  Id_Localidad  \
0             1  BUENOS AIRES           1  25 de Mayo             1   
1             1  BUENOS AIRES           1  25 de Mayo             1   
2             1  BUENOS AIRES           1  25 de Mayo             1   
3             1  BUENOS AIRES           1  25 de Mayo             1   
4             1  BUENOS AIRES           1  25 de Mayo             1   

    Localidad    Tecnologia Link Indec  Accesos  
0  25 de Mayo          ADSL    6854100    755.0  
1  25 de Mayo    CABLEMODEM    6854100   4600.0  
2  25 de Mayo  FIBRA OPT

C:\Users\jrgsi\AppData\Local\Temp\ipykernel_18828\2316679060.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Accesos'].fillna(0, inplace=True)


ETL Dial-BAf.csv

Cargamos el data set Dial-BAf.csv y realizamos una exploración para ver datos faltantes ,su estructura y visualizar las primeras líneas

In [16]:
# Cargamos el archivo CSV
archivo= r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Dial-BAf.csv'
print(f"Análisis para el archivo: {archivo}") 
df = pd.read_csv(archivo)

 # Realizamos el analisis  de el archivo
print(df.head())         
print(df.info())        
print(df.describe())     
print(f"Dimensiones del dataframe: {df.shape}")

# Análisamos el porsentaje de valores nulos valores nulos
print("Valores nulos por columna")
print(df.isnull().sum())

# Reemplazamos valores faltantes con 0 o corregir errores
df.fillna(0, inplace=True)


# Contamos el valores faltantes
missing_values = df.isnull().mean() * 100
print("Porcentaje de valores faltantes por columna:\n", missing_values)



Análisis para el archivo: H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Dial-BAf.csv
    Año  Trimestre        Provincia  Banda ancha fija  Dial up    Total
0  2024          2     Buenos Aires           4983480   5900.0  4989380
1  2024          2  Capital Federal           1461549   2050.0  1463599
2  2024          2        Catamarca             75073      1.0    75074
3  2024          2            Chaco            147087      5.0   147092
4  2024          2           Chubut            173366    857.0   174223
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1992 entries, 0 to 1991
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Año               1992 non-null   int64  
 1   Trimestre         1992 non-null   int64  
 2   Provincia         1992 non-null   object 
 3   Banda ancha fija  1992 non-null   int64  
 4   Dial up           1988 non-null   float64
 5   Total             1992 non-null   

Cambiamos el formato de fecha para que sea un formato mas compatible con Python uniendo la columna fecha con la columna trimestre, ademas se crea la columna Id_Provincia

In [17]:
# Cargamos el archivo CSV
archivo = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Dial-BAf.csv'
df = pd.read_csv(archivo)

# Mapeamos los trimestres al último día de cada trimestre
trimestre_map = {1: "03-31", 2: "06-30", 3: "09-30", 4: "12-31"}

# Convertimos 'Año' a cadena y mapear 'Trimestre' al último día del trimestre
if 'Año' in df.columns and 'Trimestre' in df.columns:
    df['Año'] = df['Año'].astype(str)
    df['Trimestre'] = df['Trimestre'].map(trimestre_map)

    # Creamos la columna 'Fecha' combinando 'Año' y 'Trimestre'
    df['Fecha'] = pd.to_datetime(df['Año'] + "-" + df['Trimestre'])

    # Borramos  las columnas 'Año' y 'Trimestre'
    df.drop(columns=['Año', 'Trimestre'], inplace=True)

    # Reordenamos las columnas 
    cols = ['Fecha'] + [col for col in df.columns if col != 'Fecha']
    df = df[cols]
else:
    print("Error: Las columnas 'Año' y 'Trimestre' no se encontraron en los datos.")


# Identificamos todas las provincias en el dataset
provincias = df['Provincia'].unique()
print("Provincias en Argentina:", provincias)
print("Número total de provincias:", len(provincias))

# Creamos un diccionario de ID únicos para cada provincia, comenzando desde 1
provincia_id_map = {provincia: idx + 1 for idx, provincia in enumerate(provincias)}

# Creamos la columna Id_Provincia usando el diccionario
df['Id_Provincia'] = df['Provincia'].map(provincia_id_map)

# Ordenamos el dataset según el orden de columnas
column_order = ['Id_Provincia','Provincia', 'Fecha','Banda ancha fija','Dial up','Total']
df = df[column_order]

# Guardamos el DataFrame final en un archivo CSV
output_path = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Dial-BAf.csv'
df.to_csv(output_path, index=False)

print(df.head())

Provincias en Argentina: ['Buenos Aires' 'Capital Federal' 'Catamarca' 'Chaco' 'Chubut' 'Córdoba'
 'Corrientes' 'Entre Ríos' 'Formosa' 'Jujuy' 'La Pampa' 'La Rioja'
 'Mendoza' 'Misiones' 'Neuquén' 'Río Negro' 'Salta' 'San Juan' 'San Luis'
 'Santa Cruz' 'Santa Fe' 'Santiago Del Estero' 'Tierra Del Fuego'
 'Tucumán']
Número total de provincias: 24
   Id_Provincia        Provincia      Fecha  Banda ancha fija  Dial up  \
0             1     Buenos Aires 2024-06-30           4983480   5900.0   
1             2  Capital Federal 2024-06-30           1461549   2050.0   
2             3        Catamarca 2024-06-30             75073      1.0   
3             4            Chaco 2024-06-30            147087      5.0   
4             5           Chubut 2024-06-30            173366    857.0   

     Total  
0  4989380  
1  1463599  
2    75074  
3   147092  
4   174223  


ETL Ingresos .csv

Cargamos el data set Ingresos .csv y realizamos una exploración para ver datos faltantes ,su estructura y visualizar las primeras líneas

In [19]:
# Cargamos el archivo CSV
archivo= r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Ingresos .csv'
print(f"Análisis para el archivo: {archivo}")  
df = pd.read_csv(archivo)

 # Realizamos el analisis  de el archivo
print(df.head())         
print(df.info())        
print(df.describe())     
print(f"Dimensiones del dataframe: {df.shape}")



# Análisamos el porsentaje de valores nulos valores nulos
print("Valores nulos por columna")
print(df.isnull().sum())

# Contamos el valores faltantes
missing_values = df.isnull().mean() * 100
print("Porcentaje de valores faltantes por columna:\n", missing_values)

# Contamos el número de filas duplicadas
num_duplicates = df.duplicated(keep=False).sum()
print("Número total de filas duplicadas:", num_duplicates)
print("\n" + "="*50 + "\n")  # Separador para cada archivo

Análisis para el archivo: H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Ingresos .csv
    Año  Trimestre  Ingresos (miles de pesos)        Periodo
0  2024          2                442032166.7   Abr-Jun 2024
1  2024          1                346198986.1   Ene-Mar 2024
2  2023          4                167376014.8   Oct-Dic 2023
3  2023          3                133106593.4  Jul-Sept 2023
4  2023          2                118060280.3   Jun-Mar 2023
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Año                        42 non-null     int64  
 1   Trimestre                  42 non-null     int64  
 2   Ingresos (miles de pesos)  42 non-null     float64
 3   Periodo                    42 non-null     object 
dtypes: float64(1), int64(2), object(1)
memory usage: 1.4+ KB
None
         Año  Trimestre  Ingresos 

Cambiamos el formato de fecha para que sea un formato más compatible con Python uniendo la columna fecha con la columna trimestre

In [ ]:
import pandas as pd

# Cargamos el archivo CSV
archivo = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Ingresos .csv'
df = pd.read_csv(archivo)

# Convertir 'Año_Temporal' a tipo entero si no lo está
df['Año_Temporal'] = df['Año_Temporal'].astype(int)

# Agrupamos por año y calculamos el ingreso anual total
ingresos_anuales = df.groupby('Año_Temporal')['Ingresos (miles de pesos)'].sum().reset_index()
ingresos_anuales.rename(columns={'Ingresos (miles de pesos)': 'Ingresos Anuales'}, inplace=True)

# Calculamos el porcentaje de cambio respecto al año anterior
ingresos_anuales['Ganancia Anual (%)'] = ingresos_anuales['Ingresos Anuales'].pct_change() * 100

# Redondear el porcentaje a dos decimales
ingresos_anuales['Ganancia Anual (%)'] = ingresos_anuales['Ganancia Anual (%)'].round(2)

# Unimos los ingresos anuales calculados al DataFrame original
df = df.merge(ingresos_anuales[['Año_Temporal', 'Ganancia Anual (%)']], on='Año_Temporal', how='left')

# Guardamos el DataFrame final en un archivo CSV
output_path = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Ingresos_Calculado.csv'
df.to_csv(output_path, index=False)

# Mostrar las primeras filas del DataFrame
print(df.head())



        Fecha  Ingresos (miles de pesos)  Año_Temporal  Ganancia Anual (%)
0  2024-06-30                442032166.7          2024                88.3
1  2024-03-31                346198986.1          2024                88.3
2  2023-12-31                167376014.8          2023                66.0
3  2023-09-30                133106593.4          2023                66.0
4  2023-06-30                118060280.3          2023                66.0


ETL Penetracion-hogares.csv

Cargamos el data set Penetracion-hogares.csv y realizamos una exploración para ver datos faltantes ,su estructura y visualizar las primeras líneas

In [36]:

# Cargamos la carpeta con el archivo .CSV 
archivo= r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Penetracion-hogares.csv'
print(f"Análisis para el archivo: {archivo}") 
df = pd.read_csv(archivo)

 # Realizamos el analisis  de el archivo
print(df.head())         
print(df.info())        
print(df.describe())     
print(f"Dimensiones del dataframe: {df.shape}")

# Análisamos el porsentaje de valores nulos valores nulos
print("Valores nulos por columna")
print(df.isnull().sum())

# Contamos el valores faltantes
missing_values = df.isnull().mean() * 100
print("Porcentaje de valores faltantes por columna:\n", missing_values)

# Contamos el número de filas duplicadas
num_duplicates = df.duplicated(keep=False).sum()
print("Número total de filas duplicadas:", num_duplicates)
print("\n" + "="*50 + "\n")  # Separador para cada archivo

Análisis para el archivo: H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Penetracion-hogares.csv
    Año  Trimestre        Provincia  Accesos por cada 100 hogares
0  2024          2     Buenos Aires                          79.8
1  2024          2  Capital Federal                         116.4
2  2024          2        Catamarca                          68.8
3  2024          2            Chaco                          44.1
4  2024          2           Chubut                          86.3
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Año                           1000 non-null   int64  
 1   Trimestre                     1000 non-null   int64  
 2   Provincia                     1000 non-null   object 
 3   Accesos por cada 100 hogares  1000 non-null   float64
dtypes: float64(1), int64(2), object(1)


Cambiamos el formato de fecha para que sea un formato más compatible con Python uniendo la columna fecha con la columna trimestre y se crea la columna Id_Provincias

In [37]:
# Cargamos el archivo CSV
archivo = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Penetracion-hogares.csv'
df = pd.read_csv(archivo)

# Mapeamos los trimestres al último día de cada trimestre
trimestre_map = {1: "03-31", 2: "06-30", 3: "09-30", 4: "12-31"}

# Convertimos 'Año' a cadena y mapear 'Trimestre' al último día del trimestre
if 'Año' in df.columns and 'Trimestre' in df.columns:
    df['Año'] = df['Año'].astype(str)
    df['Trimestre'] = df['Trimestre'].map(trimestre_map)

    # Creamos la columna 'Fecha' combinando 'Año' y 'Trimestre'
    df['Fecha'] = pd.to_datetime(df['Año'] + "-" + df['Trimestre'])

    # Borramos  las columnas 'Año' y 'Trimestre'
    df.drop(columns=['Año', 'Trimestre'], inplace=True)

    # Reordenamos las columnas 
    cols = ['Fecha'] + [col for col in df.columns if col != 'Fecha']
    df = df[cols]
else:
    print("Error: Las columnas 'Año' y 'Trimestre' no se encontraron en los datos.")
# Identificamos todas las provincias en el dataset
provincias = df['Provincia'].unique()
print("Provincias en Argentina:", provincias)
print("Número total de provincias:", len(provincias))

# Creamos un diccionario de ID únicos para cada provincia, comenzando desde 1
provincia_id_map = {provincia: idx + 1 for idx, provincia in enumerate(provincias)}

# Creamos la columna Id_Provincia usando el diccionario
df['Id_Provincia'] = df['Provincia'].map(provincia_id_map)

# Ordenamos el dataset según el orden de columnas
column_order = ['Id_Provincia','Provincia','Fecha' ,'Accesos por cada 100 hogares']
df = df[column_order]
# Guardamos el DataFrame final en un archivo CSV
output_path = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Penetracion-hogares.csv'
df.to_csv(output_path, index=False)

print(df.head())


Provincias en Argentina: ['Buenos Aires' 'Capital Federal' 'Catamarca' 'Chaco' 'Chubut' 'Córdoba'
 'Corrientes' 'Entre Ríos' 'Formosa' 'Jujuy' 'La Pampa' 'La Rioja'
 'Mendoza' 'Misiones' 'Neuquén' 'Río Negro' 'Salta' 'San Juan' 'San Luis'
 'Santa Cruz' 'Santa Fe' 'Santiago Del Estero' 'Tierra Del Fuego'
 'Tucumán']
Número total de provincias: 24
   Id_Provincia        Provincia      Fecha  Accesos por cada 100 hogares
0             1     Buenos Aires 2024-06-30                          79.8
1             2  Capital Federal 2024-06-30                         116.4
2             3        Catamarca 2024-06-30                          68.8
3             4            Chaco 2024-06-30                          44.1
4             5           Chubut 2024-06-30                          86.3


ETL Penetración-poblacion.csv

Cargamos el data set Penetración-poblacion.csv y realizamos una exploración para ver datos faltantes ,su estructura y visualizar las primeras líneas

In [38]:
# Cargamos el archivo CSV
archivo= r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Penetración-poblacion.csv'
print(f"Análisis para el archivo: {archivo}")  
df = pd.read_csv(archivo)

 # Realizamos el analisis  de el archivo
print(df.head())         
print(df.info())        
print(df.describe())     
print(f"Dimensiones del dataframe: {df.shape}")

# Análisamos el porsentaje de valores nulos valores nulos
print("Valores nulos por columna")
print(df.isnull().sum())

# Contamos el valores faltantes
missing_values = df.isnull().mean() * 100
print("Porcentaje de valores faltantes por columna:\n", missing_values)

# Contamos el número de filas duplicadas
num_duplicates = df.duplicated(keep=False).sum()
print("Número total de filas duplicadas:", num_duplicates)
print("\n" + "="*50 + "\n")  # Separador para cada archivo

Análisis para el archivo: H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Penetración-poblacion.csv
    Año  Trimestre        Provincia  Accesos por cada 100 hab
0  2024          2     Buenos Aires                      27.4
1  2024          2  Capital Federal                      47.4
2  2024          2        Catamarca                      17.5
3  2024          2            Chaco                      11.8
4  2024          2           Chubut                      26.5
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Año                       1000 non-null   int64  
 1   Trimestre                 1000 non-null   int64  
 2   Provincia                 1000 non-null   object 
 3   Accesos por cada 100 hab  1000 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 31.4+ KB
None
         Año  Trim

Cambiamos el formato de fecha para que sea un formato mas compatible con Python uniendo la columna fecha con la columna trimestre , ademas se crea la columna Id_Provincia

In [39]:
# Cargamos el archivo CSV
archivo = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Penetración-poblacion.csv'
df = pd.read_csv(archivo)

# Mapeamos los trimestres al último día de cada trimestre
trimestre_map = {1: "03-31", 2: "06-30", 3: "09-30", 4: "12-31"}

# Convertimos 'Año' a cadena y mapear 'Trimestre' al último día del trimestre
if 'Año' in df.columns and 'Trimestre' in df.columns:
    df['Año'] = df['Año'].astype(str)
    df['Trimestre'] = df['Trimestre'].map(trimestre_map)

    # Creamos la columna 'Fecha' combinando 'Año' y 'Trimestre'
    df['Fecha'] = pd.to_datetime(df['Año'] + "-" + df['Trimestre'])

    # Borramos  las columnas 'Año' y 'Trimestre'
    df.drop(columns=['Año', 'Trimestre'], inplace=True)

    # Reordenamos las columnas 
    cols = ['Fecha'] + [col for col in df.columns if col != 'Fecha']
    df = df[cols]
else:
    print("Error: Las columnas 'Año' y 'Trimestre' no se encontraron en los datos.")


# Identificamos todas las provincias en el dataset
provincias = df['Provincia'].unique()
print("Provincias en Argentina:", provincias)
print("Número total de provincias:", len(provincias))

# Creamos un diccionario de ID únicos para cada provincia, comenzando desde 1
provincia_id_map = {provincia: idx + 1 for idx, provincia in enumerate(provincias)}

# Creamos la columna Id_Provincia usando el diccionario
df['Id_Provincia'] = df['Provincia'].map(provincia_id_map)

# Ordenamos el dataset según el orden de columnas
column_order = ['Id_Provincia','Provincia', 'Fecha','Accesos por cada 100 hab']
df = df[column_order]

# Guardamos el DataFrame final en un archivo CSV
output_path = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Penetración-poblacion.csv'
df.to_csv(output_path, index=False)
print(df.head())

Provincias en Argentina: ['Buenos Aires' 'Capital Federal' 'Catamarca' 'Chaco' 'Chubut' 'Córdoba'
 'Corrientes' 'Entre Ríos' 'Formosa' 'Jujuy' 'La Pampa' 'La Rioja'
 'Mendoza' 'Misiones' 'Neuquén' 'Río Negro' 'Salta' 'San Juan' 'San Luis'
 'Santa Cruz' 'Santa Fe' 'Santiago Del Estero' 'Tierra Del Fuego'
 'Tucumán']
Número total de provincias: 24
   Id_Provincia        Provincia      Fecha  Accesos por cada 100 hab
0             1     Buenos Aires 2024-06-30                      27.4
1             2  Capital Federal 2024-06-30                      47.4
2             3        Catamarca 2024-06-30                      17.5
3             4            Chaco 2024-06-30                      11.8
4             5           Chubut 2024-06-30                      26.5


ETL Penetracion-totales.csv

Cargamos el data set Penetracion-totales.csv y realizamos una exploración para ver datos faltantes ,su estructura y visualizar las primeras líneas

In [40]:
# Cargamos el archivo CSV
archivo= r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Penetracion-totales.csv'
print(f"Análisis para el archivo: {archivo}") 
df = pd.read_csv(archivo)

 # Realizamos el analisis  de el archivo
print(df.head())         
print(df.info())        
print(df.describe())     
print(f"Dimensiones del dataframe: {df.shape}")

# Análisamos el porsentaje de valores nulos valores nulos
print("Valores nulos por columna")
print(df.isnull().sum())

# Contamos el valores faltantes
missing_values = df.isnull().mean() * 100
print("Porcentaje de valores faltantes por columna:\n", missing_values)

# Contamos el número de filas duplicadas
num_duplicates = df.duplicated(keep=False).sum()
print("Número total de filas duplicadas:", num_duplicates)
print("\n" + "="*50 + "\n")  # Separador para cada archivo

Análisis para el archivo: H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Penetracion-totales.csv
    Año  Trimestre  Accesos por cada 100 hogares  Accesos por cada 100 hab  \
0  2024          2                          78.1                      24.6   
1  2024          1                          78.9                      24.8   
2  2023          4                          78.6                      24.7   
3  2023          3                          77.8                      24.4   
4  2023          2                          77.0                      24.1   

         Periodo  
0   Abr-Jun 2024  
1   Ene-Mar 2024  
2   Oct-Dic 2023  
3  Jul-Sept 2023  
4   Abr-Jun 2023  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Año                           42 non-null     int64  
 1   Trimestre                   

Cambiamos el formato de fecha para que sea un formato más compatible con Python uniendo la columna fecha con la columna trimestre y se bora la columna Periodo

In [41]:
# Cargamos el archivo CSV
archivo = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Penetracion-totales.csv'
df = pd.read_csv(archivo)

# Mapeamos los trimestres al último día de cada trimestre
trimestre_map = {1: "03-31", 2: "06-30", 3: "09-30", 4: "12-31"}

# Convertimos 'Año' a cadena y mapear 'Trimestre' al último día del trimestre
if 'Año' in df.columns and 'Trimestre' in df.columns:
    df['Año'] = df['Año'].astype(str)
    df['Trimestre'] = df['Trimestre'].map(trimestre_map)

    # Creamos la columna 'Fecha' combinando 'Año' y 'Trimestre'
    df['Fecha'] = pd.to_datetime(df['Año'] + "-" + df['Trimestre'])

    # Borramos  las columnas 'Año' y 'Trimestre'
    df.drop(columns=['Año', 'Trimestre'], inplace=True)

    # Reordenamos las columnas 
    cols = ['Fecha'] + [col for col in df.columns if col != 'Fecha']
    df = df[cols]
else:
    print("Error: Las columnas 'Año' y 'Trimestre' no se encontraron en los datos.")

    # Eliminamos columa Periodo
df.drop(columns=['Periodo'], inplace=True)

# Guardamos el DataFrame final en un archivo CSV
output_path = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Penetracion-totales.csv'
df.to_csv(output_path, index=False)

print(df.head())

       Fecha  Accesos por cada 100 hogares  Accesos por cada 100 hab
0 2024-06-30                          78.1                      24.6
1 2024-03-31                          78.9                      24.8
2 2023-12-31                          78.6                      24.7
3 2023-09-30                          77.8                      24.4
4 2023-06-30                          77.0                      24.1


ETL Totales Accesos por rango.csv

Cargamos el data set Totales Accesos por rango.csv y realizamos una exploración para ver datos faltantes ,su estructura y visualizar las primeras líneas

In [42]:
# Cargamos el archivo CSV
archivo= r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Totales Accesos por rango.csv'
print(f"Análisis para el archivo: {archivo}")
df = pd.read_csv(archivo)

 # Realizamos el analisis  de el archivo
print(df.head())         
print(df.info())        
print(df.describe())     
print(f"Dimensiones del dataframe: {df.shape}")

# Análisamos el porsentaje de valores nulos valores nulos
print("Valores nulos por columna")
print(df.isnull().sum())

# Contamos el valores faltantes
missing_values = df.isnull().mean() * 100
print("Porcentaje de valores faltantes por columna:\n", missing_values)

# Contamos el número de filas duplicadas
num_duplicates = df.duplicated(keep=False).sum()
print("Número total de filas duplicadas:", num_duplicates)
print("\n" + "="*50 + "\n")  # Separador para cada archivo

Análisis para el archivo: H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Totales Accesos por rango.csv
    Año  Trimestre  Hasta 512 kbps  Entre 512 Kbps y 1 Mbps  \
0  2024          2           28151                    67024   
1  2024          1           28801                    69355   
2  2023          4           29708                    71742   
3  2023          3           30274                    80609   
4  2023          2           30827                    79405   

   Entre 1 Mbps y 6 Mbps  Entre 6 Mbps y 10 Mbps  Entre 10 Mbps y 20 Mbps  \
0                 840200                  911374                   662649   
1                 866152                  950930                   672155   
2                 900253                  978108                   697232   
3                 945608                  998725                   720084   
4                 963510                 1028250                   734125   

   Entre 20 Mbps y 30 Mbps  Más de 30 Mbps   OT

Cambiamos el formato de fecha para que sea un formato más compatible con Python uniendo la columna fecha con la columna trimestre 

In [43]:
# Cargamos el archivo CSV
archivo = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Totales Accesos por rango.csv'
df = pd.read_csv(archivo)

# Mapeamos los trimestres al último día de cada trimestre
trimestre_map = {1: "03-31", 2: "06-30", 3: "09-30", 4: "12-31"}

# Convertimos 'Año' a cadena y mapear 'Trimestre' al último día del trimestre
if 'Año' in df.columns and 'Trimestre' in df.columns:
    df['Año'] = df['Año'].astype(str)
    df['Trimestre'] = df['Trimestre'].map(trimestre_map)

    # Creamos la columna 'Fecha' combinando 'Año' y 'Trimestre'
    df['Fecha'] = pd.to_datetime(df['Año'] + "-" + df['Trimestre'])

    # Borramos  las columnas 'Año' y 'Trimestre'
    df.drop(columns=['Año', 'Trimestre'], inplace=True)

    # Reordenamos las columnas 
    cols = ['Fecha'] + [col for col in df.columns if col != 'Fecha']
    df = df[cols]
else:
    print("Error: Las columnas 'Año' y 'Trimestre' no se encontraron en los datos.")


# Guardamos el DataFrame final en un archivo CSV
output_path = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Totales Accesos por rango.csv'
df.to_csv(output_path, index=False)

print(df.head())

       Fecha  Hasta 512 kbps  Entre 512 Kbps y 1 Mbps  Entre 1 Mbps y 6 Mbps  \
0 2024-06-30           28151                    67024                 840200   
1 2024-03-31           28801                    69355                 866152   
2 2023-12-31           29708                    71742                 900253   
3 2023-09-30           30274                    80609                 945608   
4 2023-06-30           30827                    79405                 963510   

   Entre 6 Mbps y 10 Mbps  Entre 10 Mbps y 20 Mbps  Entre 20 Mbps y 30 Mbps  \
0                  911374                   662649                   348253   
1                  950930                   672155                   353896   
2                  978108                   697232                   350290   
3                  998725                   720084                   350918   
4                 1028250                   734125                   364730   

   Más de 30 Mbps   OTROS     Total  
0     

ETL Totales Accesos Por Tecnología.csv

Cargamos el data set Accesos Por Tecnología.csv y realizamos una exploración para ver datos faltantes ,su estructura y visualizar las primeras líneas

In [44]:
# Cargamos el archivo CSV
archivo= r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Totales Accesos Por Tecnología.csv'
print(f"Análisis para el archivo: {archivo}")  
df = pd.read_csv(archivo)

 # Realizamos el analisis  de el archivo
print(df.head())         
print(df.info())        
print(df.describe())     
print(f"Dimensiones del dataframe: {df.shape}")

# Análisamos el porsentaje de valores nulos valores nulos
print("Valores nulos por columna")
print(df.isnull().sum())

# Contamos el valores faltantes
missing_values = df.isnull().mean() * 100
print("Porcentaje de valores faltantes por columna:\n", missing_values)

# Contamos el número de filas duplicadas
num_duplicates = df.duplicated(keep=False).sum()
print("Número total de filas duplicadas:", num_duplicates)
print("\n" + "="*50 + "\n")  # Separador para cada archivo

Análisis para el archivo: H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Totales Accesos Por Tecnología.csv
    Año  Trimestre     ADSL  Cablemodem  Fibra óptica  Wireless   Otros  \
0  2024          2   733491     5867504       4169958    593197  191957   
1  2024          1   774475     5986957       4015101    598682  257941   
2  2023          4   836390     6022532       3908183    585760  194796   
3  2023          3   897895     6018832       3708718    581436  200027   
4  2023          2  1006509     5997149       3463988    581823  202428   

      Total        Periodo  
0  11556107   Abr-Jun 2024  
1  11633156   Ene-Mar 2024  
2  11547661   Oct-Dic 2023  
3  11406908  Jul-Sept 2023  
4  11251897   Abr-Jun 2023  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Año           42 non-null     int64 
 1   Trimestre     42 non-null 

Cambiamos el formato de fecha para que sea un formato más compatible con Python uniendo la columna fecha con la columna trimestre y se borra la columna Periodo.

In [45]:
# Cargamos el archivo CSV
archivo = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Totales Accesos Por Tecnología.csv'
df = pd.read_csv(archivo)

# Mapeamos los trimestres al último día de cada trimestre
trimestre_map = {1: "03-31", 2: "06-30", 3: "09-30", 4: "12-31"}

# Convertimos 'Año' a cadena y mapear 'Trimestre' al último día del trimestre
if 'Año' in df.columns and 'Trimestre' in df.columns:
    df['Año'] = df['Año'].astype(str)
    df['Trimestre'] = df['Trimestre'].map(trimestre_map)

    # Creamos la columna 'Fecha' combinando 'Año' y 'Trimestre'
    df['Fecha'] = pd.to_datetime(df['Año'] + "-" + df['Trimestre'])

    # Borramos  las columnas 'Año' y 'Trimestre'
    df.drop(columns=['Año', 'Trimestre'], inplace=True)

    # Reordenamos las columnas 
    cols = ['Fecha'] + [col for col in df.columns if col != 'Fecha']
    df = df[cols]
else:
    print("Error: Las columnas 'Año' y 'Trimestre' no se encontraron en los datos.")

# Eliminamos columnas poco relevantes
df.drop(columns=['Periodo'], inplace=True)

# Guardamos el DataFrame final en un archivo CSV
output_path = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Totales Accesos Por Tecnología.csv'
df.to_csv(output_path, index=False)

print(df.head())


       Fecha     ADSL  Cablemodem  Fibra óptica  Wireless   Otros     Total
0 2024-06-30   733491     5867504       4169958    593197  191957  11556107
1 2024-03-31   774475     5986957       4015101    598682  257941  11633156
2 2023-12-31   836390     6022532       3908183    585760  194796  11547661
3 2023-09-30   897895     6018832       3708718    581436  200027  11406908
4 2023-06-30  1006509     5997149       3463988    581823  202428  11251897


ETL Totales Dial-BAf.csv

Cargamos el data set Totales Dial-BAf.csv y realizamos una exploración para ver datos faltantes ,su estructura y visualizar las primeras líneas

In [46]:
# Cargamos el archivo CSV
archivo= r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Totales Dial-BAf.csv'
print(f"Análisis para el archivo: {archivo}") 
df = pd.read_csv(archivo)

 # Realizamos el analisis  de el archivo
print(df.head())         
print(df.info())        
print(df.describe())     
print(f"Dimensiones del dataframe: {df.shape}")

# Análisamos el porsentaje de valores nulos valores nulos
print("Valores nulos por columna")
print(df.isnull().sum())

# Contamos el valores faltantes
missing_values = df.isnull().mean() * 100
print("Porcentaje de valores faltantes por columna:\n", missing_values)

# Contamos el número de filas duplicadas
num_duplicates = df.duplicated(keep=False).sum()
print("Número total de filas duplicadas:", num_duplicates)
print("\n" + "="*50 + "\n")  # Separador para cada archivo

Análisis para el archivo: H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Totales Dial-BAf.csv
    Año  Trimestre  Banda ancha fija  Dial up     Total        Periodo
0  2024          2          11544314    11793  11556107   Abr-Jun 2024
1  2024          1          11621363    11793  11633156   Ene-Mar 2024
2  2023          4          11535868    11793  11547661   Oct-Dic 2023
3  2023          3          11395115    11793  11406908  Jul-Sept 2023
4  2023          2          11239781    12116  11251897   Abr-Jun 2023
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Año               42 non-null     int64 
 1   Trimestre         42 non-null     int64 
 2   Banda ancha fija  42 non-null     int64 
 3   Dial up           42 non-null     int64 
 4   Total             42 non-null     int64 
 5   Periodo           42 non-null     object
dt

Cambiamos el formato de fecha para que sea un formato más compatible con Python uniendo la columna fecha con la columna trimestre.

In [47]:
# Cargamos el archivo CSV
archivo = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Totales Dial-BAf.csv'
df = pd.read_csv(archivo)

# Mapeamos los trimestres al último día de cada trimestre
trimestre_map = {1: "03-31", 2: "06-30", 3: "09-30", 4: "12-31"}

# Convertimos 'Año' a cadena y mapear 'Trimestre' al último día del trimestre
if 'Año' in df.columns and 'Trimestre' in df.columns:
    df['Año'] = df['Año'].astype(str)
    df['Trimestre'] = df['Trimestre'].map(trimestre_map)

    # Creamos la columna 'Fecha' combinando 'Año' y 'Trimestre'
    df['Fecha'] = pd.to_datetime(df['Año'] + "-" + df['Trimestre'])

    # Borramos  las columnas 'Año' y 'Trimestre'
    df.drop(columns=['Año', 'Trimestre'], inplace=True)

    # Reordenamos las columnas 
    cols = ['Fecha'] + [col for col in df.columns if col != 'Fecha']
    df = df[cols]
else:
    print("Error: Las columnas 'Año' y 'Trimestre' no se encontraron en los datos.")

    # Eliminamos columna Periodo
df.drop(columns=['Periodo'], inplace=True)

# Guardamos el DataFrame final en un archivo CSV
output_path = r'C:/Users/jrgsi/OneDrive/Escritorio/PROUECTOS HENRY/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Totales Dial-BAf.csv'
df.to_csv(output_path, index=False)

print(df.head())

       Fecha  Banda ancha fija  Dial up     Total
0 2024-06-30          11544314    11793  11556107
1 2024-03-31          11621363    11793  11633156
2 2023-12-31          11535868    11793  11547661
3 2023-09-30          11395115    11793  11406908
4 2023-06-30          11239781    12116  11251897


ETL Totales VMD.csv

Cargamos el data set Totales VMD.csv y realizamos una exploración para ver datos faltantes ,su estructura y visualizar las primeras líneas

In [48]:
# Cargamos el archivo CSV
archivo= r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Totales VMD.csv'
print(f"Análisis para el archivo: {archivo}")  

 # Realizamos el analisis  de el archivo
print(df.head())         
print(df.info())        
print(df.describe())     
print(f"Dimensiones del dataframe: {df.shape}")

# Análisamos el porsentaje de valores nulos valores nulos
print("Valores nulos por columna")
print(df.isnull().sum())

# Contamos el valores faltantes
missing_values = df.isnull().mean() * 100
print("Porcentaje de valores faltantes por columna:\n", missing_values)

# Contamos el número de filas duplicadas
num_duplicates = df.duplicated(keep=False).sum()
print("Número total de filas duplicadas:", num_duplicates)
print("\n" + "="*50 + "\n")  # Separador para cada archivo

Análisis para el archivo: H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Totales VMD.csv
       Fecha  Banda ancha fija  Dial up     Total
0 2024-06-30          11544314    11793  11556107
1 2024-03-31          11621363    11793  11633156
2 2023-12-31          11535868    11793  11547661
3 2023-09-30          11395115    11793  11406908
4 2023-06-30          11239781    12116  11251897
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Fecha             42 non-null     datetime64[ns]
 1   Banda ancha fija  42 non-null     int64         
 2   Dial up           42 non-null     int64         
 3   Total             42 non-null     int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 1.4 KB
None
                               Fecha  Banda ancha fija  Dial up      Total
count                             42    

Cambiamos el formato de fecha para que sea un formato más compatible con Python uniendo la columna fecha con la columna trimestre y eleminamos la columna Trimestre1

In [49]:
# Cargamos el archivo CSV
archivo = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Totales VMD.csv'
df = pd.read_csv(archivo)

# Mapeamos los trimestres al último día de cada trimestre
trimestre_map = {1: "03-31", 2: "06-30", 3: "09-30", 4: "12-31"}

# Convertimos 'Año' a cadena y mapear 'Trimestre' al último día del trimestre
if 'Año' in df.columns and 'Trimestre' in df.columns:
    df['Año'] = df['Año'].astype(str)
    df['Trimestre'] = df['Trimestre'].map(trimestre_map)

    # Creamos la columna 'Fecha' combinando 'Año' y 'Trimestre'
    df['Fecha'] = pd.to_datetime(df['Año'] + "-" + df['Trimestre'])

    # Borramos  las columnas 'Año' y 'Trimestre'
    df.drop(columns=['Año', 'Trimestre'], inplace=True)

    # Reordenamos las columnas 
    cols = ['Fecha'] + [col for col in df.columns if col != 'Fecha']
    df = df[cols]
else:
    print("Error: Las columnas 'Año' y 'Trimestre' no se encontraron en los datos.")

# Eliminamos la columna Trimestre.1 
df.drop(columns=['Trimestre.1'], inplace=True)

#Redondeamos la columna Mbps a un decimal
df['Mbps (Media de bajada)'] = df['Mbps (Media de bajada)'].round(1)

  # Guardamos el DataFrame final en un archivo CSV
output_path = r'C:/Users/jrgsi/OneDrive/Escritorio/PROUECTOS HENRY/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Totales VMD.csv'
df.to_csv(output_path, index=False)

print(df.head())  

       Fecha  Mbps (Media de bajada)
0 2024-06-30                   139.2
1 2024-03-31                   139.2
2 2023-12-31                   139.0
3 2023-09-30                   129.7
4 2023-06-30                   124.0


ETL Velocidad % por prov.csv

Cargamos el data set Velocidad % por prov.csv y realizamos una exploración para ver datos faltantes ,su estructura y visualizar las primeras líneas

In [50]:
# Cargamos el archivo CSV
archivo= r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Velocidad % por prov.csv'
print(f"Análisis para el archivo: {archivo}") 
df = pd.read_csv(archivo)

 # Realizamos el analisis  de el archivo
print(df.head())         
print(df.info())        
print(df.describe())     
print(f"Dimensiones del dataframe: {df.shape}")

# Análisamos el porsentaje de valores nulos valores nulos
print("Valores nulos por columna")
print(df.isnull().sum())

# Contamos el valores faltantes
missing_values = df.isnull().mean() * 100
print("Porcentaje de valores faltantes por columna:\n", missing_values)

# Contamos el número de filas duplicadas
num_duplicates = df.duplicated(keep=False).sum()
print("Número total de filas duplicadas:", num_duplicates)
print("\n" + "="*50 + "\n")  # Separador para cada archivo

Análisis para el archivo: H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Velocidad % por prov.csv
    Año  Trimestre        Provincia  Mbps (Media de bajada)
0  2024          2     Buenos Aires                   157.4
1  2024          2  Capital Federal                   233.0
2  2024          2        Catamarca                    97.4
3  2024          2            Chaco                   107.8
4  2024          2           Chubut                    21.7
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Año                     1008 non-null   int64  
 1   Trimestre               1008 non-null   int64  
 2   Provincia               1008 non-null   object 
 3   Mbps (Media de bajada)  1008 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 31.6+ KB
None
         Año  Trimestre  Mbps (Media de ba

Cambiamos el formato de fecha para que sea un formato más compatible con Python uniendo la columna fecha con la columna trimestre y se crea la columna Id_Provincias

In [51]:
# Cargamos el archivo CSV
archivo = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Velocidad % por prov.csv'
df = pd.read_csv(archivo)

# Mapeamos los trimestres al último día de cada trimestre
trimestre_map = {1: "03-31", 2: "06-30", 3: "09-30", 4: "12-31"}

# Convertimos 'Año' a cadena y mapear 'Trimestre' al último día del trimestre
if 'Año' in df.columns and 'Trimestre' in df.columns:
    df['Año'] = df['Año'].astype(str)
    df['Trimestre'] = df['Trimestre'].map(trimestre_map)

    # Creamos la columna 'Fecha' combinando 'Año' y 'Trimestre'
    df['Fecha'] = pd.to_datetime(df['Año'] + "-" + df['Trimestre'])

    # Borramos  las columnas 'Año' y 'Trimestre'
    df.drop(columns=['Año', 'Trimestre'], inplace=True)

    # Reordenamos las columnas 
    cols = ['Fecha'] + [col for col in df.columns if col != 'Fecha']
    df = df[cols]
else:
    print("Error: Las columnas 'Año' y 'Trimestre' no se encontraron en los datos.")


# Identificamos todas las provincias en el dataset
provincias = df['Provincia'].unique()
print("Provincias en Argentina:", provincias)
print("Número total de provincias:", len(provincias))

# Creamos un diccionario de ID únicos para cada provincia, comenzando desde 1
provincia_id_map = {provincia: idx + 1 for idx, provincia in enumerate(provincias)}

# Creamos la columna Id_Provincia usando el diccionario
df['Id_Provincia'] = df['Provincia'].map(provincia_id_map)

# Ordenamos el dataset según el orden de columnas
column_order = ['Id_Provincia','Provincia', 'Fecha','Mbps (Media de bajada)']
df = df[column_order]

# Guardamos el DataFrame final en un archivo CSV
output_path = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Velocidad % por prov.csv.csv'
df.to_csv(output_path, index=False)

print(df.head())

Provincias en Argentina: ['Buenos Aires' 'Capital Federal' 'Catamarca' 'Chaco' 'Chubut' 'Córdoba'
 'Corrientes' 'Entre Ríos' 'Formosa' 'Jujuy' 'La Pampa' 'La Rioja'
 'Mendoza' 'Misiones' 'Neuquén' 'Río Negro' 'Salta' 'San Juan' 'San Luis'
 'Santa Cruz' 'Santa Fe' 'Santiago Del Estero' 'Tierra Del Fuego'
 'Tucumán']
Número total de provincias: 24
   Id_Provincia        Provincia      Fecha  Mbps (Media de bajada)
0             1     Buenos Aires 2024-06-30                   157.4
1             2  Capital Federal 2024-06-30                   233.0
2             3        Catamarca 2024-06-30                    97.4
3             4            Chaco 2024-06-30                   107.8
4             5           Chubut 2024-06-30                    21.7


ETL Velocidad_sin_Rangos.csv

Cargamos el data set Velocidad_sin_Rangos.csv y realizamos una exploración para ver datos faltantes ,su estructura y visualizar las primeras líneas

In [52]:
# Cargamos el archivo 
archivo= r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Velocidad_sin_Rangos.csv'
print(f"Análisis para el archivo: {archivo}")  
df = pd.read_csv(archivo)

 # Realizamos el analisis  de el archivo
print(df.head())         
print(df.info())        
print(df.describe())     
print(f"Dimensiones del dataframe: {df.shape}")

# Análisamos el porsentaje de valores nulos valores nulos
print("Valores nulos por columna")
print(df.isnull().sum())

# Contamos el valores faltantes
missing_values = df.isnull().mean() * 100
print("Porcentaje de valores faltantes por columna:\n", missing_values)

# Contamos el número de filas duplicadas
num_duplicates = df.duplicated(keep=False).sum()
print("Número total de filas duplicadas:", num_duplicates)
print("\n" + "="*50 + "\n")  # Separador para cada archivo

Análisis para el archivo: H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Velocidad_sin_Rangos.csv
    Año  Trimestre     Provincia  Velocidad  Accesos
0  2024          2  BUENOS AIRES       75.0     1062
1  2024          2  BUENOS AIRES       59.0       59
2  2024          2  BUENOS AIRES      480.0        5
3  2024          2  BUENOS AIRES        3.5    41735
4  2024          2  BUENOS AIRES       18.0     1042
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18884 entries, 0 to 18883
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Año        18884 non-null  int64  
 1   Trimestre  18884 non-null  int64  
 2   Provincia  18884 non-null  object 
 3   Velocidad  18874 non-null  float64
 4   Accesos    18884 non-null  int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 737.8+ KB
None
          Año  Trimestre  Velocidad   Accesos
count 18884.0    18884.0    18874.0   18884.0
mean   2021.0        2.5   

In [53]:
# Cargamos el archivo CSV
archivo = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Velocidad_sin_Rangos.csv'
df = pd.read_csv(archivo)

# Mapeamos los trimestres al último día de cada trimestre
trimestre_map = {1: "03-31", 2: "06-30", 3: "09-30", 4: "12-31"}

# Convertimos 'Año' a cadena y mapear 'Trimestre' al último día del trimestre
if 'Año' in df.columns and 'Trimestre' in df.columns:
    df['Año'] = df['Año'].astype(str)
    df['Trimestre'] = df['Trimestre'].map(trimestre_map)

    # Creamos la columna 'Fecha' combinando 'Año' y 'Trimestre'
    df['Fecha'] = pd.to_datetime(df['Año'] + "-" + df['Trimestre'])

    # Borramos  las columnas 'Año' y 'Trimestre'
    df.drop(columns=['Año', 'Trimestre'], inplace=True)

    # Reordenamos las columnas 
    cols = ['Fecha'] + [col for col in df.columns if col != 'Fecha']
    df = df[cols]
else:
    print("Error: Las columnas 'Año' y 'Trimestre' no se encontraron en los datos.")


# Identificamos todas las provincias en el dataset
provincias = df['Provincia'].unique()
print("Provincias en Argentina:", provincias)
print("Número total de provincias:", len(provincias))

# Creamos un diccionario de ID únicos para cada provincia, comenzando desde 1
provincia_id_map = {provincia: idx + 1 for idx, provincia in enumerate(provincias)}

# Creamos la columna Id_Provincia usando el diccionario
df['Id_Provincia'] = df['Provincia'].map(provincia_id_map)

# Ordenamos el dataset según el orden de columnas
column_order = ['Id_Provincia','Provincia', 'Fecha','Velocidad','Accesos']

# Guardamos el DataFrame final en un archivo CSV
output_path = r'H:/PROYECTO INDIVIDUAL DOS/datos/dataset .csv/Internet/Velocidad_sin_Rangos.csv'
df.to_csv(output_path, index=False)

print(df.head())

Provincias en Argentina: ['BUENOS AIRES' 'CABA' 'CATAMARCA' 'CHACO' 'CHUBUT' 'CORDOBA' 'CORRIENTES'
 'ENTRE RIOS' 'FORMOSA' 'JUJUY' 'LA PAMPA' 'LA RIOJA' 'MENDOZA' 'MISIONES'
 'NEUQUEN' 'RIO NEGRO' 'SALTA' 'SAN JUAN' 'SAN LUIS' 'SANTA CRUZ'
 'SANTA FE' 'SANTIAGO DEL ESTERO' 'TIERRA DEL FUEGO' 'TUCUMAN'
 'Buenos Aires' 'Capital Federal' 'Catamarca' 'Chaco' 'Chubut' 'Córdoba'
 'Corrientes' 'Entre Ríos' 'Formosa' 'Jujuy' 'La Pampa' 'La Rioja'
 'Mendoza' 'Misiones' 'Neuquén' 'Río Negro' 'Salta' 'San Juan' 'San Luis'
 'Santa Cruz' 'Santa Fe' 'Santiago Del Estero' 'Tierra Del Fuego'
 'Tucumán']
Número total de provincias: 48
       Fecha     Provincia  Velocidad  Accesos  Id_Provincia
0 2024-06-30  BUENOS AIRES       75.0     1062             1
1 2024-06-30  BUENOS AIRES       59.0       59             1
2 2024-06-30  BUENOS AIRES      480.0        5             1
3 2024-06-30  BUENOS AIRES        3.5    41735             1
4 2024-06-30  BUENOS AIRES       18.0     1042             1
